# Esercitazione 3 - Teoria delle valenze di Hanks

Studenti:

- Brunello Matteo (mat. 858867)
- Caresio Lorenzo (mat. 836021)

*Consegna*: si richiede un'implementazione della teoria sulle valenze di Patrick Hanks. In particolare, partendo da un corpus a scelta e uno specifico verbo (tendenzialmente non troppo frequente e/o generico ma nemmeno raro), l'idea è di costruire dei possibili cluster semantici, con relativa frequenza. Ad es. dato il verbo "to see" con valenza = 2, e usando un parser sintattico (ad es. Spacy), si possono collezionare eventuali fillers per i ruoli di *subj* e *obj* del verbo, per poi convertirli in semantic types. Un cluster frequente su "*to see*" potrebbe unire *subj = noun.person* con *obj = noun.artifact*. Si richiede di partire da un corpus di almeno alcune centinaia di istanze del verbo.

In [42]:
import collections

from nltk import word_tokenize
from nltk.corpus import brown, reuters, wordnet, gutenberg
from nltk.wsd import lesk
from nltk.stem import WordNetLemmatizer
import nltk

# Lemmatization and Word Senses
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

# Corpora
nltk.download('brown')
nltk.download('reuters')
nltk.download('gutenberg')

# Parser
import spacy
spcy = spacy.load("en_core_web_sm") # For accuracy over efficiency use en_core_web_trf

# WSD
%pip install -q pywsd
from pywsd.lesk import adapted_lesk

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


### Verbo utilizzato

Per lo svolgimento e l'analisi dell'implementazione si è deciso di utilizzare il verbo *to cut* con valenza pari a 2 (tranisitivo), abbastanza generico ma il cui utilizzo può essere sia concreto sia metaforico. Si permette di parametrizzare sia il verbo che la sua valenza.

In [74]:
 # PARAMETERS
verb = lemmatizer.lemmatize('cut')
valence = 2

### Raccolta e filtraggio del dataset

Come dataset si è deciso di unire tre corpora, il Brown, il Reuters e quello del Progetto Gutenberg (già integrati in NLTK), così da ottenere un dataset iniziale più ampio e più variegato (così da idenitificare più *significati* finali e più variegati).

In [75]:
corpus_sents = []

for sentence in brown.sents():
  corpus_sents.append(sentence)
for sentence in reuters.sents():
  corpus_sents.append(sentence)
for sentence in gutenberg.sents():
  corpus_sents.append(sentence)

len(corpus_sents)

210608

Dalle duecentodiecimila frasi ottenute si filtrano quelle che contengono (in versione lemmatizata) il termine *cut*.

In [76]:
def filter_sentences(verb, sentences):
  filtered_sentences = []

  for sentence in sentences:
    if verb in [lemmatizer.lemmatize(word) for word in sentence]:
      filtered_sentences.append(sentence)

  return filtered_sentences

In [77]:
filtered_sents = filter_sentences(verb, corpus_sents)
len(filtered_sents)

1669

Già in questa fase si sarebbe potuto eseguire il parsing delle frasi del dataset attraverso Spacy, selezionando quindi le frasi in cui il termine ricercato agisca effettivamente da verbo. Questo avrebbe però richiesto di eseguire il parsing completo delle oltre duecentomila frasi contenute nel dataset, un'operazione particolarmente esosa.

Si è invece scelto di andare a lemmatizzare le frasi del corpus e a verificare che queste contenessero il lemma del verbo desiderato. Si è passato quindi da una richiesta temporale di svariati minuti a una di circa $20$ secondi, passando inoltre da duecentodiecimila frasi a $1669$ per il verbo *to cut*. Si trattano però di frasi in cui il lemma del verbo compare, non è però che detto che questi svolgano effettivamente il ruolo di verbo (potrebbero infatti, per esempio, svolgere il ruolo di sostantivi o verbi sostantivati).

È quindi comunque necessario fare un parsing delle frasi filtrate, ma come anticipato lo si farà nell'ordine del migliaio e non nelle centinaia di migliaia.

In [78]:
# Function needed to stip pointless whitespaces around punctuation
def get_processed_sent(list_of_words):
  sentence = " ".join(list_of_words)
  sequence_to_replace = [' / ', ' . ', ' .', ' , ', ' - ', ' \' ', '( ', ' )', ' :']
  for seq in sequence_to_replace:
    sentence = sentence.replace(seq, seq.strip())
  return sentence

def parse_and_filter_sents(verb, sentences):
  filtered_sentences = []

  for sentence in sentences:
    parsed_sentence = spcy(get_processed_sent(sentence)) # Use Spacy for parsing and lemmatizing
    if (verb, 'VERB') in [(w.lemma_, w.pos_) for w in parsed_sentence]:
      filtered_sentences.append(parsed_sentence)

  return filtered_sentences

In [79]:
parsed_sents = parse_and_filter_sents(verb, filtered_sents)
len(parsed_sents)

1224

Le frasi in cui *cut* compare effettivamente come verbo sono quindi $1224$, il $73\%$ delle 1669 filtrate (dalle duecentodiecimila iniziali).

### Estrazione degli argomenti del verbo

In base alla valenza, si va a estrarre gli argomenti del verbo *cut*, da cui verranno poi estratti i tipi semantici.

Si sfrutta il parsing a dipendenze di Spacy per discriminare sulla funzione sintattica dei vari termini (il loro ruolo come *soggetto* o *oggetto*) *figli* del verbo, andando a discriminare sul tipo della dipendenza (vari tipo di *soggetto*, *oggetto diretto* e *dativo* per i verbi transitivi, *oggetto indiretto* o *oggetto passivo* per i verbi ditransitivi).

In [80]:
def get_verb_arguments(verb, sentences):

  verb_arguments = []

  for sent in sentences:
    for word in sent:
      if word.lemma_ == verb and word.pos_ == 'VERB':

        subjs = [child for child in word.children if 'subj' in child.dep_]

        if valence > 1:
          objs = [child for child in word.children if child.dep_ == 'dobj' or child.dep_ == 'dative']
        if valence == 3: # Add indirect objects to objs list (to cover ditransitive verb)
          objs.extend([child for child in word.children if child.dep_ == 'iobj' or child.dep_ == 'pobj'])

        # Collect the found arguments according to the valence
        if valence == 1 and len(subjs) > 0:
          for subj in subjs:
              verb_arguments.append((sent, subj))
        elif valence > 1 and len(subjs) > 0 and len(objs) > 0:
          for subj in subjs:
            for obj in objs:
              verb_arguments.append((sent, subj, obj))

  return verb_arguments

In [81]:
arguments = get_verb_arguments(verb, parsed_sents)
print(f"{len(arguments)} arguments derived from {len(set([argument[0] for argument in arguments]))} sentences.")

419 arguments derived from 390 sentences.


Il numero di frasi in cui *cut* compare come verbo transitivo è quindi $390$, il $23\%$ delle 1669 filtrate (dalle duecentodiecimila iniziali). È ora possibile utilizzare questi argomenti in un'applicazione della Teoria di Hanks andandone a derivare i *semantic types*.

### Teoria di Hanks

Come *semantic types* si è scelto di utilizzare i *lexnames* dei synset (come suggerito d'altronde nella consegna), i nomi dei file lessicografici in cui sono raccolti i synset. I *lexnames* relativi ai sostantivi sono $25$ (consultabili [qui](https://wordnet.princeton.edu/documentation/lexnames5wn)).

Dapprima si era adottata un'implementazione minimale basata sull'utilizzo del *Simple Lesk* fornito da NLTK, dopo aver constatato delle prestazioni sub-ottimali si è deciso prima di utilizzare una libreria apposita per la WSD (`pywsd`) con versioni di Lesk più avanzate, e poi di gestire manualmente i pronomi la cui risoluzione è banale (*i*, *he*, *she*, *we*). Non si gestiscono invece i casi restanti (si veda più avanti), molto più ambigui.


In [82]:
def get_semantic_type(word, sentence):
  # Explicit handling of easily-resolvable pronouns
  if word.pos_ == 'PRON' and word.lemma_.lower() in ['i', 'he', 'she']: return 'noun.person'
  elif word.pos_ == 'PRON' and word.lemma_.lower() == 'we': return 'noun.group'
  else:
      synset = adapted_lesk(str(sentence), str(word), 'n') # Use Adapted Lex to do WSD
      if synset: return synset.lexname() # If a synset is found, use its lexname as Semantic Type
  return None

def get_semantic_types(arguments):
  semantic_types = []

  for argument in arguments: # argument = (sent, subj) if valence == 1, (sent, subj, obj) if valence > 1

    subj_st = get_semantic_type(argument[1], argument[0])
    if valence > 1: obj_st = get_semantic_type(argument[2], argument[0])

    # Add semantic types only if they're actually found
    if valence == 1 and subj_st: semantic_types.append([argument[1], subj_st])
    elif valence > 1 and subj_st and obj_st: semantic_types.append([argument[1], subj_st, argument[2], obj_st])

  return semantic_types

In [83]:
computed_semantic_types = get_semantic_types(arguments)

In base alla valenza, per ogni semantic type o coppia di semantic types si raccolgono i fillers (lemmatizzati) da visualizzare successivamente.

In [84]:
# Compute a list of fillers for each semantic types (valence = 1) or for each semantic types combination (valence > 1)
subjs_dict = {}
if valence > 1: objs_dict= {}

if valence == 1: # Collect fillers for each subj semantic type
  subjs_dict = dict((subj, []) for subj in [semantic_types[1] for semantic_types in computed_semantic_types])
  for semtantic_types in computed_semantic_types:
    subjs_dict[semtantic_types[1]].append(semtantic_types[0].lemma_.lower())

elif valence > 1: # Collect fillers for each subj-obj semantic types combination
  for (subj_st, obj_st) in [(couple[1], couple[3]) for couple in computed_semantic_types]:
    subjs_dict[(subj_st, obj_st)]= []
    objs_dict[(subj_st, obj_st)] = []
  for semtantic_types in computed_semantic_types:
    subjs_dict[(semtantic_types[1], semtantic_types[3])].append(semtantic_types[0].lemma_.lower())
    objs_dict[(semtantic_types[1], semtantic_types[3])].append(semtantic_types[2].lemma_.lower())

Infine si va quindi a contare le frequenze delle varie combinazioni di tipi semantici (i *significati*) e li si ordina per frequenza (visualizzando anche i filler lemmatizzati che in essi hanno rappresentato gli argomenti del verbo).

In [85]:
if valence == 1: meanings = [st[1] for st in computed_semantic_types]
elif valence > 1: meanings = [(st[1], st[3]) for st in computed_semantic_types]

meanings_with_frequency = collections.Counter(meanings).items() # Use Counter to get frequency for each meaning
sorted_meanings = sorted(meanings_with_frequency, key=lambda st: st[1], reverse=True) # Sort meanings by frequency

# Print meanings and their lemmatized fillers
print(f"Identified meanings sorted by frequency ({len(sorted_meanings)} meanings):")

for meaning, freq in sorted_meanings:

  if valence == 1: print(f" {meaning} [{verb}] ({freq} times)")
  elif valence > 1: print(f" {meaning[0]} [{verb}] {meaning[1]} ({freq} times)")

  if valence == 1:
    print(f"   Subjects: {set(subjs_dict[meaning])}")
  elif valence > 1:
    print(f"   Subjects: {set(subjs_dict[(meaning[0], meaning[1])])}")
    print(f"   Objects: {set(objs_dict[(meaning[0], meaning[1])])}")

Identified meanings sorted by frequency (105 meanings):
 noun.cognition [cut] noun.possession (30 times)
   Subjects: {'it'}
   Objects: {'cost', 'expense', 'stake', 'tariff', 'price', 'dividend', 'capital', 'surcharge', 'rate', 'deficit'}
 noun.person [cut] noun.person (18 times)
   Subjects: {'father', 'stranger', 'jezebel', 'mayer', 'she', 'he', 'i'}
   Objects: {'man', 'chicken', 'prophet', 'tree', 'inhabitant', 'nobles', 'judge', 'doer', 'price', 'she', 'he', 'i', 'spirit'}
 noun.person [cut] noun.artifact (14 times)
   Subjects: {'rebel', 'he', 'i', 'she'}
   Objects: {'bond', 'horse', 'shoe', 'engine', 'cord', 'sprig', 'chariot', 'skirt', 'image', 'ram', 'line'}
 noun.person [cut] noun.cognition (13 times)
   Subjects: {'he', 'carpenter', 'i'}
   Objects: {'it', 'witchcraft', 'memory', 'one'}
 noun.person [cut] noun.group (9 times)
   Subjects: {'wife', 'economists', 'lord', 'she', 'he', 'i'}
   Objects: {'pair', 'thicket', 'institute', 'city', 'nation', 'country', 'multitude'}


La presenza di svariati *significati* della forma `noun.cognition [cut]` o `noun.cognition [cut] X`  dominati dal soggetto *it* evidenzia un problema fondamentale: la necessità di una fase di risoluzione delle [coreferenze](https://w.wiki/8WXz). La quasi totalità di questi *it* infatti fa riferimento ad altri componenti delle frasi, non accessibili direttamente però tramite un *semplice* parsing a dipendenze come quello adottato.

 Si tratta però di un fenomeno la cui risoluzione non è banale: si potrebbe ignorare le occorrenze di *it* e dei vari pronomi ambigui (modificando quindi `get_semantic_type`), oppure, più correttamente, sarebbe necessario inserire una fase di risoluzione delle coreferenze nel parsing di SpaCy (possibile, ma ancora in fase sperimentale). In questa sede ci limitano invece semplicemente a evidenziare questa criticità.